# Multi Head Attention

### Terminos nuevos:
* h - heads (cada cabeza representa una capa de paralelizacion de nuestro scaled dot-product attention)

El multi-head attention es una version escalada del self dot-product attention que se vio anteriormente, solo que este utiliza varias capas corriendo en paralelo.

<img src="./images/multi_head_attention.png" alt="scaled_dot_prod_attent" width="200" height="auto"> <img src="./images/mha_formula.png" alt="scaled_dot_prod_attent" width="500" height="auto">

In [3]:
import torch
import torch.nn as nn

In [4]:
phrase = "Hola mi nombre es Alex"
input_phrase = phrase.split(' ')

In [5]:
sequence_len = len(input_phrase)
batch_size = 1
input_dim = 512
d_model = 512
x = torch.randn((batch_size, sequence_len, input_dim))

In [58]:
x.size()

torch.Size([1, 5, 512])

In [7]:
qkv_layer = nn.Linear(input_dim, 3*d_model)

In [8]:
qkv = qkv_layer(x)

In [9]:
qkv.size()

torch.Size([1, 5, 1536])

In [10]:
h = 8
h_dim = d_model // h
qkv = qkv.reshape(batch_size, sequence_len, h, 3*h_dim)

In [11]:
qkv.size()

torch.Size([1, 5, 8, 192])

In [12]:
qkv = qkv.permute(0,2,1,3) #batch, num_heads, sequence_len, 3*head_dim

In [13]:
qkv.size()

torch.Size([1, 8, 5, 192])

In [33]:
q, k, v = qkv.chunk(3, dim=-1)
q.shape

torch.Size([1, 8, 5, 64])

Una vez que tenemos separados nuestros datos en multi-heads para q, k y v, ahora vamos a utilizar las funciones que se crearon de para calcular la atencion por cada una de nuestras heads.

<img src="./images/attention.png" alt="attention formula" width="400" height="auto">

In [48]:
import math
import torch.nn.functional as F

def scaled_dot_product(Q, V, K, Use_Mask=False):
    d_k = K.shape[-1]
    scaled = torch.matmul(Q, K.transpose(-1,-2))/math.sqrt(d_k)
    if Use_Mask:
        mask = torch.full(scaled.size(), float('-inf'))
        mask = torch.triu(mask, diagonal=1)
        scaled += mask
    attention = F.softmax(scaled, dim=-1)
    out = torch.matmul(attention, V)
    return out

In [50]:
res = scaled_dot_product(Q=q, V=v, K=k, Use_Mask=True)

In [51]:
res.shape

torch.Size([1, 8, 5, 64])

Con este vector de salida que nos arrojo la funcion de scaled_dot_product tenemos que los valores corresponden a:
1 -> batch size
8 -> heads
5 -> sequence len
64 -> head_dim

Por lo que ahora toca concatenar todos los valores de las heads de nuestro modelo.

In [52]:
concat = res.reshape(batch_size, sequence_len, h*h_dim)

In [54]:
concat.shape

torch.Size([1, 5, 512])

Teniendo esto, ahora toca el ultimo paso de multi-head attention que es pasar el resultado a una funcion lineal.

In [55]:
linear_layer = nn.Linear(d_model, d_model)

In [56]:
out = linear_layer(concat)

In [57]:
out.shape

torch.Size([1, 5, 512])